# pFGW Framework for Feature Matching & Tracking

In [1]:
import sys
import os
import networkx as nx
import gc
import time

sys.path.append("../GWMT/")

In [2]:
from GWMT import *
import readMergeTree as rmt

## Data Input 

This is an example to read our preset datasets. Our merge tree data is split into two files: ``treeEdges_monoMesh_*.txt`` and ``treeNodes_monoMesh_*.txt``, representing the data for tree edges and tree nodes, respectively.

To read your own dataset, your tree data should be saved in a ``nx.Graph`` object, in which each node has properties for its spacial coordinates (e.g., "x", "y"), its scalar value (e.g., "height"), and its critical type (0: minimum, 1: saddle, 2: maximum. This setting cannot be changed). 
Besides, you also need to provide the node id for the root node of the tree.

The tree data should be stored in a ``GWMergeTree`` object.

==========================================================

Our tree edge data format:

a_0, b_0  
a_1, b_1  
...  
a_{|E|-1}, b_{|E|-1}

Each row describes the indices of two nodes that the edge connecting in between. Edges are undirected.

==========================================================

Our tree node data format:

x_0, y_0, z_0, scalar_0, type_0  
x_1, y_1, z_1, scalar_1, type_1  
...  
x_{|V|-1}, y_{|V|-1}, z_{|V|-1}, scalar_{|V|-1}, type_{|V|-1}

Each row has five components: the "x", "y", "z" coordinates, the scalar value, and the critical point type for the node.


In [3]:
datasets = [
            "CPPin20230801_0percent",
            # "CPPin20230802_0percent", 
            # "CPPin20230803_0percent",
            # "CPPin20230804_0percent", 
            # "CPPin20230805_0percent", 
            # "CPPin20230806_0percent",
            # "CPPin20230807_0percent", 
            # "CPPin20230808_0percent",
           ]

We merged several processing steps in the following code block.

This is because we need to remove intermediate variables using garbage collection (gc).
If the variables exist in multiple blocks, it is hard to release their memory even after we delete them in Jupyter notebook.

In [4]:
maxima_only = True
value_thres = 2.0

mt_list = dict()
root_list = dict()

# Let's not oversimplify the merge tree, because now we need many nodes as anchor points

# This serves for removing the very-small cloud system from the results entirely
# This should be very small
disappear_volume_threshold = 10

# This is to reduce the number of anchor points (but not remove) for cloud systems
# This can be a bit large
volume_threshold = 30

simplified_mt_list = dict()
simplified_root_list = dict()

# Loading the dataset (Merge tree)
for dataset in datasets:
    print("Working on", dataset)
    dataset_path = os.path.join("../data", dataset)
    mt_list[dataset] = []
    root_list[dataset] = []
    
    for froot, di, files in os.walk(dataset_path):

        def key(s):
            try:
                int(s)
                return int(s)
            except ValueError:
                return len(files) + 1

        def endsWithTxt(s: str):
            return s.endswith("txt")

        def endsWithNpy(s: str):
            return s.endswith("npy")

        txt_files = list(filter(endsWithTxt, files))
        txt_files.sort(key=lambda x: key(x.split(".")[0].split("_")[-1]))

        # You need to specify the root node type. Choices: ["minimum", "maximum"]
        # (Avoid specifying merge tree type to avoid confusion between split tree and join tree in different contexts)
        for file in txt_files:
            trees, roots = rmt.get_trees(os.path.join(dataset_path, file), root_type="minimum", threshold=value_thres)
            mt_list[dataset].extend(trees)
            root_list[dataset].extend(roots)

    assert (len(root_list[dataset]) == len(mt_list[dataset]))

# For benchmark usage only
max_raw_nodes = 0
max_simplified_nodes = 0
total_raw_nodes = 0
total_simplified_nodes = 0
n_runs = 0
mt_simplification_runtime = 0

# We simplify merge trees with volume thresholds
for dataset in datasets:
    print("Working on", dataset)
    simplified_mt_list[dataset] = [None for i in range(len(mt_list[dataset]))]
    simplified_root_list[dataset] = [None for i in range(len(mt_list[dataset]))]

    idx = 0

    for mt in mt_list[dataset]:
        if volume_threshold > 0:
            # For benchmark purpose
            simplify_t0 = time.perf_counter()
            
            _, simp_mt = volume_simplify_mt(mt, 
                                            vol_thres=volume_threshold, 
                                            disappear_vol_thres=disappear_volume_threshold, 
                                            vol_name="volume", 
                                            stop_saddle_val=value_thres)
            
            # For benchmark purpose
            simplify_dt = time.perf_counter() - simplify_t0
            n_runs += 1
            mt_simplification_runtime += simplify_dt
            
            simplified_mt_list[dataset][idx] = simp_mt
            simplified_root_list[dataset][idx] = simp_mt.root
            
            max_raw_nodes = max(max_raw_nodes, mt.number_of_nodes())
            max_simplified_nodes = max(max_simplified_nodes, simp_mt.number_of_nodes())
            total_raw_nodes += mt.number_of_nodes()
            total_simplified_nodes += simp_mt.number_of_nodes()
            
        else:
            simplified_mt_list[dataset][idx] = mt
            simplified_root_list[dataset][idx] = mt.root
            
        idx += 1

print("Benchmark information - Simplify Merge Tree:", max_raw_nodes, max_simplified_nodes, n_runs, mt_simplification_runtime)
        
for dataset in datasets:
    lmt = len(mt_list[dataset])
    mt_list[dataset] = [None] * lmt

# We put the simplified merge trees into the GWMergeTree object
gwmt_list = dict()

for dataset in datasets:
    print("Working on", dataset)
    gwmt_list[dataset] = [GWMergeTree(simplified_mt_list[dataset][i], simplified_root_list[dataset][i]) for i in range(len(mt_list[dataset]))]

for dataset in datasets:
    lmt = len(simplified_mt_list[dataset])
    simplified_mt_list[dataset] = [None] * lmt
    simplified_root_list[dataset] = [None] * lmt
    
gc.collect()

Working on CPPin20230801_0percent
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Adding volume to the merge tree
Working on CPPin20230801_0percent
Initially removing 30804 leaves.
Ini

1494234

In [5]:
print("Benchmark information - Simplify Merge Tree (Total #nodes):", total_raw_nodes, total_simplified_nodes)

Benchmark information - Simplify Merge Tree (Total #nodes): 1502783 57975


## Parameter Initialization

We now specify the parameters to be passed to the pFGW framework, including the following:

*spacial_scale*: a float number denoting the spacial length of the scalar field. (e.g., $max(height, width)$ for a 2D rectangular scalar field).  
         This is needed for normalizing the spacial coordinates for nodes.  
         This number can be flexible. However, it affects the optimal $\alpha$ value. Please make sure that the normalization is reasonable. A recommended range for the maximum Euclidean distance between any pair of critical points after normalization is $[0.5, 2.0]$.
         

*labels*: a list of strings denoting the spacial coordinate labels. (e.g., ["x", "y"] for a 2D scalar field)

*scalar_name*: the name of the scalar field in GWMergeTree objects.

*edge_weight_name*: the name of the weight of edges in GWMergeTree objects.

*weight_mode*: the strategy to encode $W$. Choices: ["shortestpath", "lca"].

*prob_distribution*: the strategy to encode $p$. Choices: ["uniform", "ancestor"]

In [6]:
if "CloudDensity" in dataset:
    spacial_scale = 255 
elif "CPPin202308" in dataset:
    spacial_scale = 932
elif "juelich" in dataset:
    spacial_scale = 427
    
max_value = 150
labels = ["x", "y", "z"]
scalar_name = "height"
edge_weight_name = "weight"
weight_mode = "shortestpath"
prob_distribution = "uniform"
fully_initialized = False

# lca-threshold
intrinsic_rescale = 1
if weight_mode == "lca":
    intrinsic_rescale = max_value
elif weight_mode == "shortestpath":
    intrinsic_rescale = max_value * 2
    
all_scales = [spacial_scale, spacial_scale, spacial_scale, intrinsic_rescale]

In [7]:
# Validify the GWMergeTree object. This is not mandatory, but recommended to check whether your data input format is correct
for dataset in datasets:
    print("Working on", dataset)
    for tree in gwmt_list[dataset]:
        tree.label_validation(labels, scalar_name, edge_weight_name)

Working on CPPin20230801_0percent


## Framework Initialization

Initializing the pFGW framework with given parameters and data input. There is an output for the normalization process consisting $L+1$ elements, where the first $L$ elements are $\frac{1}{rescaling\_factor}$ for coordinate labels ($L$ coordinate labels in total), and the last element is $\frac{1}{rescaling\_factor}$ for scalar function values.

In [8]:
pfgws = dict()

for dataset in datasets:
    print("Working on", dataset)
    pfgws[dataset] = GWTracking(
        gwmt_list[dataset],
        spacial_scale,
        labels,
        scalar_name=scalar_name,
        edge_weight_name=edge_weight_name,
        weight_mode=weight_mode,
        prob_distribution=prob_distribution,
        tracking_maxima_only=maxima_only,
        maxima_only=maxima_only,
        fully_initialized=fully_initialized,
        scalar_threshold=value_thres
    )

Working on CPPin20230801_0percent
WARNING! We do not compute the scale for all time steps.
Please provide the scale for normalization when intialize steps.


In [9]:
for dataset in datasets:
    pfgws[dataset].set_all_scales(all_scales)

## Parameter Tuning

We present a demo for our parameter tuning process in this section.

This is not the only way for parameter tuning. We aim to optimize the one-to-one matching result in the first place.

In [10]:
import pandas as pd

# The optional parameter "amijo" determines the way for OT solver to converge, which may change the result
# Either picking False or True follows our framework pipeline. They just may reach different local optimal solutions.

amijo = False
max_dist_tuning = False

# The max_workers refers to the max number of threads you want the parallel run to have
max_dist = 0.0065
max_workers = 7

### Actual tracking with tuning the m parameter

In [11]:
finished_runs = dict()

In [12]:
# New!!! We are using auto parameter tuning strategies
# For cloud tracking project, there is a forced search range for other cloud tracking tools
# Therefore, we can safely assume the max matched distance without tuning
alpha_list = [0.4]

# m_range largely affects the runtime performance!!
# The default eps of m search space is 1e-2. 
# Therefore, you need ceil(log_2((m_range[1]-m_range[0])*100)) computations to get the best m value.
# If you can't make the difference in m_range below 0.16, then you should make it around 0.30 (e.g., [0.60, 0.90]), 
# in which case you need 5 runs to get the optimal m value.
m_range = [0.60, 0.90]

for dataset in datasets:
    print("Working on", dataset)
    timesteps = list(range(len(gwmt_list[dataset])))

    parameter_tuning_path = os.path.join("binary-parameter-tuning", "{}".format(dataset))
    os.makedirs(parameter_tuning_path, exist_ok=True)
    oc_path = os.path.join(parameter_tuning_path, "ocs")
    os.makedirs(oc_path, exist_ok=True)

    for alpha in alpha_list:
        if (dataset in finished_runs) and (alpha in finished_runs[dataset]):
            continue
        print("alpha =", alpha)
        best_ms, best_ocs, m_searchspace, dist_values, init_rt, pfgw_rt = pfgws[dataset].adaptive_m_tuning_binary_search(timesteps,
                                                                                             alpha, 
                                                                                             max_dist, 
                                                                                             amijo, 
                                                                                             m_range, 
                                                                                             max_workers, 
                                                                                             metric="l2", 
                                                                                             prob_rubric="nonzero",
                                                                                             benchmark=True)
        
        print("Initialization average runtime:", init_rt)
        print("pFGW average runtime:", pfgw_rt)

        save_binary_parameter_tuning(parameter_tuning_path, oc_path, alpha, best_ms, best_ocs, m_searchspace, dist_values)
        if dataset not in finished_runs:
            finished_runs[dataset] = [alpha]
        else:
            finished_runs[dataset].append(alpha)

Working on CPPin20230801_0percent
alpha = 0.4
working from timestep 0 to 1...
Finished intializing timestep 0 , CList shape: (1013,)
Finished intializing timestep 1 , CList shape: (1108,)
The optimal m for timestep 0 vs. 1 is 0.515.
Max Matched Dist list: [np.float64(0.00625638615326754)]
INIT runtime: [23.079040199983865, 29.82843499997398]
pFGW runtime: [65.27913959999569]
Initialization average runtime: 26.453737599978922
pFGW average runtime: 65.27913959999569


In [14]:
# We save the best-tuned results into the results folder
for dataset in datasets:
    print("Working on", dataset)
    if dataset not in finished_runs:
        continue
        
    parameter_tuning_path = os.path.join("binary-parameter-tuning", "{}".format(dataset))
    oc_path = os.path.join(parameter_tuning_path, "ocs")
    timesteps = list(range(len(gwmt_list[dataset])))

    output_root = os.path.join("./initial-output/", dataset)
    os.makedirs(output_root, exist_ok=True)
    for e, alpha in enumerate(alpha_list):
        if alpha not in finished_runs[dataset]:
            continue
            
        print("alpha = {}".format(str(alpha)))
        best_ms, best_ocs, _, _ = load_binary_parameter_tuning(parameter_tuning_path, oc_path, alpha)

        output_path = os.path.join(output_root, str(round(alpha, 1)))
        os.makedirs(output_path, exist_ok=True)

        for em, m in enumerate(best_ms):
            print(str('%.3f' % m))
            # We filter out coupling information unrelated to maxima
            # Note: the filtered coupling matrix does not sum to m
            id1 = timesteps[em]
            id2 = timesteps[em+1]

            oc = best_ocs[em]
            oc_maxima = oc # pfgw.filtered_oc(id1, id2, oc)
            np.savetxt(os.path.join(output_path, "oc_{}_{}.txt".format(str(id1), str(id2))), oc_maxima)


Working on CPPin20230801_0percent
alpha = 0.4
0.515
